In [3]:
!pip install psycopg2 config configparser

  Using cached config-0.5.0.post0-py2.py3-none-any.whl (20 kB)
  Using cached configparser-5.0.0-py3-none-any.whl (22 kB)


In [6]:
import pandas as pd
import numpy as np
import psycopg2
import config
from configparser import ConfigParser

In [8]:
df = pd.read_excel('Input ClosureTable.xlsx')
df.head()

,Total,Division,Company,Brand,Product Group,Department,Productsubgroup,Product,Product_CD
0,Total,Division 1,MyCompanyA,SuperDuper,Grocery,Fruits,Sweet Fruits,Apple,Apple_1kg
1,Total,Division 1,MyCompanyA,Mars,Choco,Bars,Mars packages,Mars 100g bar,Mars_100g_0815
2,Total,Division 1,MyCompanyB,Cola,Foo,Bars,Foo2,Foo,Foo_0815
3,Total,Division 1,MyCompanyB,Sunny,Grocery,Fruits,Sweet Fruits,Banana,Banana_1kg
4,Total,Division 1,MyCompanyB,Sunny,Grocery,Fruits,Sweet Fruits,Strawberries,Strawberries_100g


In [ ]:
def config(filename, section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

def connect(config_file='config.ini'):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config(filename='/project/config.ini')

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
        
        # create a cursor
        cur = conn.cursor()
        
    # execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')

        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)
       
    # close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')
            
def create_tables(config_file='config.ini'):
    """ create tables in the PostgreSQL database"""
    commands = (
        """
        CREATE TABLE vendors (
            vendor_id SERIAL PRIMARY KEY,
            vendor_name VARCHAR(255) NOT NULL
        )
        """,
        """ CREATE TABLE parts (
                part_id SERIAL PRIMARY KEY,
                part_name VARCHAR(255) NOT NULL
                )
        """,
        """
        CREATE TABLE part_drawings (
                part_id INTEGER PRIMARY KEY,
                file_extension VARCHAR(5) NOT NULL,
                drawing_data BYTEA NOT NULL,
                FOREIGN KEY (part_id)
                REFERENCES parts (part_id)
                ON UPDATE CASCADE ON DELETE CASCADE
        )
        """,
        """
        CREATE TABLE vendor_parts (
                vendor_id INTEGER NOT NULL,
                part_id INTEGER NOT NULL,
                PRIMARY KEY (vendor_id , part_id),
                FOREIGN KEY (vendor_id)
                    REFERENCES vendors (vendor_id)
                    ON UPDATE CASCADE ON DELETE CASCADE,
                FOREIGN KEY (part_id)
                    REFERENCES parts (part_id)
                    ON UPDATE CASCADE ON DELETE CASCADE
        )
        """)
    conn = None
    try:
        # read the connection parameters
        params = config(filename='/project/config.ini')
        # connect to the PostgreSQL server
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        # create table one by one
        for command in commands:
            cur.execute(command)
        # close communication with the PostgreSQL database server
        cur.close()
        # commit the changes
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
